In [6]:
from urllib.request import *
from statistics import *
from bs4 import BeautifulSoup


def league_tables_scraping(season_list,url):
    team_dict = {}
    for season in season_list:
        #url = 'https://int.soccerway.com/national/england/premier-league/%s/' % season
        #url = 'https://int.soccerway.com/national/germany/bundesliga/%s/' % season
        full_url = url+season
        try:
            page = urlopen(full_url)
        except:
            new_season = "".join(season.split("-", 2)[:2])
            #url = 'https://int.soccerway.com/national/england/premier-league/%s/' % season
            #url = 'https://int.soccerway.com/national/germany/bundesliga/%s/' % season
            full_url = url+new_season
            page = urlopen(full_url)
        soup = BeautifulSoup(page, 'html.parser')
        table = soup.find_all("table", {"class": "leaguetable sortable table detailed-table"})
        table_row = table[0].tbody.find_all('tr')
        for row in table_row:
            rank = row.find_all("td", {"class": "rank"})[0].string
            team = row.find_all("td", {"class": "team"})[0].string
            if team not in team_dict:
                team_dict[team] = {}
            team_dict[team][season] = rank
    return team_dict
    
def find_median_league_position(league_dict):
    median_position_dict = {}
    for key in league_dict:
        median_position_dict[key] = {}
        season_list = []
        for season in league_dict[key]:
            season_list.append(int(league_dict[key][season]))
            # until-season includes the current season e.g
            # until-2011-2012 uncludes season 2011-2012
            median_position_dict[key]['until-'+season]=median(season_list)
    return median_position_dict

def assign_to_classes(team_dict):
    #class 0 too weak
    #class 1 weak
    #class 2 strong
    #class 3 too strong
    for team in team_dict:
        for season in team_dict[team]:
            if team_dict[team][season] <= 5:
                team_dict[team][season] = 3
            elif (team_dict[team][season] > 5) and (team_dict[team][season] <= 10):
                team_dict[team][season] = 2
            elif (team_dict[team][season] > 10) and (team_dict[team][season] <= 15):
                team_dict[team][season] = 1
            else:
                team_dict[team][season] = 0
    return team_dict

def adapt_names_to_kaggle_db(classified_league_dict,league):
    pl_needed_teams = ['Arsenal', 'Aston Villa', 'Birmingham City', 'Blackburn Rovers', 'Blackpool', 
                       'Bolton Wanderers', 'Bournemouth', 'Burnley', 'Cardiff City', 'Chelsea', 
                       'Crystal Palace', 'Everton', 'Fulham', 'Hull City', 'Leicester City', 'Liverpool', 
                       'Manchester City', 'Manchester United', 'Middlesbrough', 'Newcastle United', 'Norwich City', 
                       'Portsmouth', 'Queens Park Rangers', 'Reading', 'Southampton', 'Stoke City', 'Sunderland', 
                       'Swansea City', 'Tottenham Hotspur', 'Watford', 'West Bromwich Albion', 'West Ham United', 
                       'Wigan Athletic', 'Wolverhampton Wanderers']

    bund_needed_teams = ['1. FC Kaiserslautern', '1. FC Köln', '1. FC Nürnberg', '1. FSV Mainz 05', 'Bayer 04 Leverkusen', 
                         'Borussia Dortmund', 'Borussia Mönchengladbach', 'DSC Arminia Bielefeld', 'Eintracht Braunschweig', 
                         'Eintracht Frankfurt', 'FC Augsburg', 'FC Bayern Munich', 'FC Energie Cottbus', 'FC Ingolstadt 04', 
                         'FC Schalke 04', 'FC St. Pauli', 'Fortuna Düsseldorf', 'Hamburger SV', 'Hannover 96', 
                         'Hertha BSC Berlin', 'Karlsruher SC', 'SC Freiburg', 'SC Paderborn 07', 'SV Darmstadt 98', 
                         'SV Werder Bremen', 'SpVgg Greuther Fürth', 'TSG 1899 Hoffenheim', 'VfB Stuttgart', 'VfL Bochum', 
                         'VfL Wolfsburg']
    if league == 'pl':
        pairs_dict = {'Queens Park Ra…': 'Queens Park Rangers',
                 'West Bromwich …': 'West Bromwich Albion',
                 'Wolverhampton …': 'Wolverhampton Wanderers',
                 'AFC Bournemouth':'Bournemouth'}
    if league == 'bund':
        pairs_dict = {'Kaiserslautern':'1. FC Kaiserslautern',
                     'Köln':'1. FC Köln',
                     'Nürnberg':'1. FC Nürnberg',
                     'Mainz 05':'1. FSV Mainz 05',
                     'Bayer Leverkusen':'Bayer 04 Leverkusen',
                     "Borussia M'gla…":'Borussia Mönchengladbach',
                     'Arminia Bielefeld':'DSC Arminia Bielefeld',
                     'Eintracht Brau…':'Eintracht Braunschweig',
                     'Eintracht Fran…':'Eintracht Frankfurt',
                     'Augsburg':'FC Augsburg',
                     'Bayern Munich':'FC Bayern Munich',
                     'Energie Cottbus':'FC Energie Cottbus',
                     'Ingolstadt':'FC Ingolstadt 04',
                     'Schalke 04':'FC Schalke 04',
                     'St. Pauli':'FC St. Pauli',
                     'Fortuna Dusseldorf':'Fortuna Düsseldorf',
                     'Hertha BSC':'Hertha BSC Berlin',
                     'Freiburg':'SC Freiburg',
                     'Paderborn':'SC Paderborn 07',
                     'Darmstadt 98':'SV Darmstadt 98',
                     'Werder Bremen':'SV Werder Bremen',
                     'Greuther Fürth':'SpVgg Greuther Fürth',
                     'Hoffenheim':'TSG 1899 Hoffenheim',
                     'Stuttgart':'VfB Stuttgart',
                     'Bochum':'VfL Bochum',
                     'Wolfsburg':'VfL Wolfsburg'}
        
    copied_dict = classified_league_dict.copy()
    for team in classified_league_dict:
        if team in pairs_dict:
            print(team)
            copied_dict[pairs_dict[team]] = classified_league_dict[team]
            del copied_dict[team]
    
                
    return copied_dict
        
    

In [7]:
season_list = []
league_list = []
url_list = ['https://int.soccerway.com/national/england/premier-league/', 
            'https://int.soccerway.com/national/germany/bundesliga/']
for i in range(1994,2016):
    season_list.append(str(i)+"-"+str(i+1))
#for url in url_list:
#    league_list.append(league_tables_scraping(season_list,url))
#print(league_list)


In [3]:
median_league_dict = {}
leagues = ['pl','bund']
i =0
for league in league_list:
    median_league_dict[leagues[i]]=find_median_league_position(league)
    i = i+1
print(median_league_dict)

{'pl': {'Blackburn Rovers': {'until-1994-1995': 1, 'until-1995-1996': 4.0, 'until-1996-1997': 7, 'until-1997-1998': 6.5, 'until-1998-1999': 7, 'until-2001-2002': 8.5, 'until-2002-2003': 7, 'until-2003-2004': 8.5, 'until-2004-2005': 10, 'until-2005-2006': 8.5, 'until-2006-2007': 10, 'until-2007-2008': 8.5, 'until-2008-2009': 10, 'until-2009-2010': 10.0, 'until-2010-2011': 10, 'until-2011-2012': 10.0}, 'Manchester United': {'until-1994-1995': 2, 'until-1995-1996': 1.5, 'until-1996-1997': 1, 'until-1997-1998': 1.5, 'until-1998-1999': 1, 'until-1999-2000': 1.0, 'until-2000-2001': 1, 'until-2001-2002': 1.0, 'until-2002-2003': 1, 'until-2003-2004': 1.0, 'until-2004-2005': 1, 'until-2005-2006': 1.5, 'until-2006-2007': 1, 'until-2007-2008': 1.0, 'until-2008-2009': 1, 'until-2009-2010': 1.0, 'until-2010-2011': 1, 'until-2011-2012': 1.0, 'until-2012-2013': 1, 'until-2013-2014': 1.0, 'until-2014-2015': 1, 'until-2015-2016': 1.5}, 'Nottingham Forest': {'until-1994-1995': 3, 'until-1995-1996': 6.0,

In [4]:
classified_league_dict = {}
for league in median_league_dict:
    classified_league_dict[league]=assign_to_classes(median_league_dict[league].copy())
print(classified_league_dict)

{'pl': {'Blackburn Rovers': {'until-1994-1995': 3, 'until-1995-1996': 3, 'until-1996-1997': 2, 'until-1997-1998': 2, 'until-1998-1999': 2, 'until-2001-2002': 2, 'until-2002-2003': 2, 'until-2003-2004': 2, 'until-2004-2005': 2, 'until-2005-2006': 2, 'until-2006-2007': 2, 'until-2007-2008': 2, 'until-2008-2009': 2, 'until-2009-2010': 2, 'until-2010-2011': 2, 'until-2011-2012': 2}, 'Manchester United': {'until-1994-1995': 3, 'until-1995-1996': 3, 'until-1996-1997': 3, 'until-1997-1998': 3, 'until-1998-1999': 3, 'until-1999-2000': 3, 'until-2000-2001': 3, 'until-2001-2002': 3, 'until-2002-2003': 3, 'until-2003-2004': 3, 'until-2004-2005': 3, 'until-2005-2006': 3, 'until-2006-2007': 3, 'until-2007-2008': 3, 'until-2008-2009': 3, 'until-2009-2010': 3, 'until-2010-2011': 3, 'until-2011-2012': 3, 'until-2012-2013': 3, 'until-2013-2014': 3, 'until-2014-2015': 3, 'until-2015-2016': 3}, 'Nottingham Forest': {'until-1994-1995': 3, 'until-1995-1996': 2, 'until-1996-1997': 2, 'until-1998-1999': 1}, 

In [5]:
pl_needed_teams = ['Arsenal', 'Aston Villa', 'Birmingham City', 'Blackburn Rovers', 'Blackpool', 'Bolton Wanderers', 'Bournemouth', 'Burnley', 'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Hull City', 'Leicester City', 'Liverpool', 'Manchester City', 'Manchester United', 'Middlesbrough', 'Newcastle United', 'Norwich City', 'Portsmouth', 'Queens Park Rangers', 'Reading', 'Southampton', 'Stoke City', 'Sunderland', 'Swansea City', 'Tottenham Hotspur', 'Watford', 'West Bromwich Albion', 'West Ham United', 'Wigan Athletic', 'Wolverhampton Wanderers']

bund_needed_teams = ['1. FC Kaiserslautern', '1. FC Köln', '1. FC Nürnberg', '1. FSV Mainz 05', 'Bayer 04 Leverkusen', 'Borussia Dortmund', 'Borussia Mönchengladbach', 'DSC Arminia Bielefeld', 'Eintracht Braunschweig', 'Eintracht Frankfurt', 'FC Augsburg', 'FC Bayern Munich', 'FC Energie Cottbus', 'FC Ingolstadt 04', 'FC Schalke 04', 'FC St. Pauli', 'Fortuna Düsseldorf', 'Hamburger SV', 'Hannover 96', 'Hertha BSC Berlin', 'Karlsruher SC', 'SC Freiburg', 'SC Paderborn 07', 'SV Darmstadt 98', 'SV Werder Bremen', 'SpVgg Greuther Fürth', 'TSG 1899 Hoffenheim', 'VfB Stuttgart', 'VfL Bochum', 'VfL Wolfsburg']

In [15]:
adapted_league_dict = {}
adapted_league_dict['pl']=adapt_names_to_kaggle_db(classified_league_dict['pl'].copy(),'pl')
adapted_league_dict['bund']=adapt_names_to_kaggle_db(classified_league_dict['bund'].copy(),'bund')
print(adapted_league_dict)

Queens Park Ra…
West Bromwich …
Wolverhampton …
AFC Bournemouth
Werder Bremen
Freiburg
Kaiserslautern
Borussia M'gla…
Bayern Munich
Bayer Leverkusen
Eintracht Fran…
Köln
Schalke 04
Stuttgart
Bochum
Fortuna Dusseldorf
St. Pauli
Arminia Bielefeld
Hertha BSC
Wolfsburg
Nürnberg
Energie Cottbus
Mainz 05
Hoffenheim
Augsburg
Greuther Fürth
Eintracht Brau…
Paderborn
Ingolstadt
Darmstadt 98
{'pl': {'Blackburn Rovers': {'until-1994-1995': 3, 'until-1995-1996': 3, 'until-1996-1997': 2, 'until-1997-1998': 2, 'until-1998-1999': 2, 'until-2001-2002': 2, 'until-2002-2003': 2, 'until-2003-2004': 2, 'until-2004-2005': 2, 'until-2005-2006': 2, 'until-2006-2007': 2, 'until-2007-2008': 2, 'until-2008-2009': 2, 'until-2009-2010': 2, 'until-2010-2011': 2, 'until-2011-2012': 2}, 'Manchester United': {'until-1994-1995': 3, 'until-1995-1996': 3, 'until-1996-1997': 3, 'until-1997-1998': 3, 'until-1998-1999': 3, 'until-1999-2000': 3, 'until-2000-2001': 3, 'until-2001-2002': 3, 'until-2002-2003': 3, 'until-2003-2

In [16]:
pl_needed_teams = ['Arsenal', 'Aston Villa', 'Birmingham City', 'Blackburn Rovers', 'Blackpool', 'Bolton Wanderers', 'Bournemouth', 'Burnley', 'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Hull City', 'Leicester City', 'Liverpool', 'Manchester City', 'Manchester United', 'Middlesbrough', 'Newcastle United', 'Norwich City', 'Portsmouth', 'Queens Park Rangers', 'Reading', 'Southampton', 'Stoke City', 'Sunderland', 'Swansea City', 'Tottenham Hotspur', 'Watford', 'West Bromwich Albion', 'West Ham United', 'Wigan Athletic', 'Wolverhampton Wanderers']

bund_needed_teams = ['1. FC Kaiserslautern', '1. FC Köln', '1. FC Nürnberg', '1. FSV Mainz 05', 'Bayer 04 Leverkusen', 'Borussia Dortmund', 'Borussia Mönchengladbach', 'DSC Arminia Bielefeld', 'Eintracht Braunschweig', 'Eintracht Frankfurt', 'FC Augsburg', 'FC Bayern Munich', 'FC Energie Cottbus', 'FC Ingolstadt 04', 'FC Schalke 04', 'FC St. Pauli', 'Fortuna Düsseldorf', 'Hamburger SV', 'Hannover 96', 'Hertha BSC Berlin', 'Karlsruher SC', 'SC Freiburg', 'SC Paderborn 07', 'SV Darmstadt 98', 'SV Werder Bremen', 'SpVgg Greuther Fürth', 'TSG 1899 Hoffenheim', 'VfB Stuttgart', 'VfL Bochum', 'VfL Wolfsburg']

print(len(pl_needed_teams))
print(len(bund_needed_teams))

34
30


In [17]:
k = 0
for team in adapted_league_dict['pl']:
    if team in pl_needed_teams:
        print(team)
        k = k +1
print(k)

Blackburn Rovers
Manchester United
Liverpool
Newcastle United
Tottenham Hotspur
Southampton
Chelsea
Arsenal
West Ham United
Everton
Manchester City
Aston Villa
Crystal Palace
Norwich City
Leicester City
Middlesbrough
Bolton Wanderers
Sunderland
Watford
Fulham
Birmingham City
Portsmouth
Wigan Athletic
Reading
Stoke City
Hull City
Burnley
Blackpool
Swansea City
Cardiff City
Queens Park Rangers
West Bromwich Albion
Wolverhampton Wanderers
Bournemouth
34


In [18]:
adapted_league_dict['pl']

{'Arsenal': {'until-1994-1995': 1,
  'until-1995-1996': 2,
  'until-1996-1997': 3,
  'until-1997-1998': 3,
  'until-1998-1999': 3,
  'until-1999-2000': 3,
  'until-2000-2001': 3,
  'until-2001-2002': 3,
  'until-2002-2003': 3,
  'until-2003-2004': 3,
  'until-2004-2005': 3,
  'until-2005-2006': 3,
  'until-2006-2007': 3,
  'until-2007-2008': 3,
  'until-2008-2009': 3,
  'until-2009-2010': 3,
  'until-2010-2011': 3,
  'until-2011-2012': 3,
  'until-2012-2013': 3,
  'until-2013-2014': 3,
  'until-2014-2015': 3,
  'until-2015-2016': 3},
 'Aston Villa': {'until-1994-1995': 0,
  'until-1995-1996': 1,
  'until-1996-1997': 3,
  'until-1997-1998': 2,
  'until-1998-1999': 2,
  'until-1999-2000': 2,
  'until-2000-2001': 2,
  'until-2001-2002': 2,
  'until-2002-2003': 2,
  'until-2003-2004': 2,
  'until-2004-2005': 2,
  'until-2005-2006': 2,
  'until-2006-2007': 2,
  'until-2007-2008': 2,
  'until-2008-2009': 2,
  'until-2009-2010': 2,
  'until-2010-2011': 2,
  'until-2011-2012': 2,
  'until-2012

In [19]:
k = 0
for team in adapted_league_dict['bund']:
    if team in bund_needed_teams:
        print(team)
        k = k +1
print(k)

Borussia Dortmund
Karlsruher SC
Hamburger SV
Hannover 96
SV Werder Bremen
SC Freiburg
1. FC Kaiserslautern
Borussia Mönchengladbach
FC Bayern Munich
Bayer 04 Leverkusen
Eintracht Frankfurt
1. FC Köln
FC Schalke 04
VfB Stuttgart
VfL Bochum
Fortuna Düsseldorf
FC St. Pauli
DSC Arminia Bielefeld
Hertha BSC Berlin
VfL Wolfsburg
1. FC Nürnberg
FC Energie Cottbus
1. FSV Mainz 05
TSG 1899 Hoffenheim
FC Augsburg
SpVgg Greuther Fürth
Eintracht Braunschweig
SC Paderborn 07
FC Ingolstadt 04
SV Darmstadt 98
30


In [20]:
adapted_league_dict['bund']

{'1. FC Kaiserslautern': {'until-1994-1995': 3,
  'until-1995-1996': 2,
  'until-1997-1998': 3,
  'until-1998-1999': 3,
  'until-1999-2000': 3,
  'until-2000-2001': 3,
  'until-2001-2002': 3,
  'until-2002-2003': 2,
  'until-2003-2004': 2,
  'until-2004-2005': 2,
  'until-2005-2006': 2,
  'until-2010-2011': 2,
  'until-2011-2012': 2},
 '1. FC Köln': {'until-1994-1995': 2,
  'until-1995-1996': 1,
  'until-1996-1997': 2,
  'until-1997-1998': 1,
  'until-2000-2001': 2,
  'until-2001-2002': 1,
  'until-2003-2004': 1,
  'until-2005-2006': 1,
  'until-2008-2009': 1,
  'until-2009-2010': 1,
  'until-2010-2011': 1,
  'until-2011-2012': 1,
  'until-2014-2015': 1,
  'until-2015-2016': 1},
 '1. FC Nürnberg': {'until-1998-1999': 0,
  'until-2001-2002': 0,
  'until-2002-2003': 0,
  'until-2004-2005': 0,
  'until-2005-2006': 1,
  'until-2006-2007': 1,
  'until-2007-2008': 1,
  'until-2009-2010': 0,
  'until-2010-2011': 1,
  'until-2011-2012': 1,
  'until-2012-2013': 1,
  'until-2013-2014': 1},
 '1. 

In [23]:
import csv
for league in adapted_league_dict:
    with open('stength_'+league+'.csv', 'w',encoding="utf-8") as myCsvFile:
        columns = ['TeamName', 'Season', 'Strength']
        writer = csv.DictWriter(myCsvFile, fieldnames=columns)
        writer.writeheader()
        for team in adapted_league_dict[league]:
            for season in adapted_league_dict[league][team]:
                writer.writerow({'TeamName': team, 'Season': season, 'Strength': adapted_league_dict[league][team][season]})